Here we will construct the CIs from the multiple trained bootstrap models. We have trained 10 bootstraps for now, we will use these to get the required CIs as we have done earlier for a different dataset.

In [1]:
# These are the ones that is already on the train script
import os
import json
import sys
import warnings
from pathlib import Path
from pprint import pformat
from typing import Dict, Union
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
# generator related imports
from New_data_generator_with_tf import DataGenerator, batch_predict
from tensorflow.keras.utils import Sequence

# [Req] IMPROVE imports
# notice that the improvelibs are in the folder that is a level above, but in the same parent directory
sys.path.append(os.path.abspath(os.path.join('..', 'IMPROVE')))
from improvelib.applications.drug_response_prediction.config import DRPTrainConfig
from improvelib.utils import str2bool
import improvelib.utils as frm
from improvelib.metrics import compute_metrics

# Model-specific imports
from model_params_def import train_params # [Req]

2025-04-22 11:17:44.874091: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-22 11:17:45.615816: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# First we compute the model misspecification variance (estimate) using the already trained models using the train dataset.

In [3]:
# For computing the PIs we also need to estimate the variance of the errors. Once we compute the above, we can follow the steps in the paper "Constructing Optimal Predictio Intervals by Using Neural Networks and Bootstrap Method" (we follow their section II and not their proposed method as it seems a little complex to implement)by Khosravi et al. in 2015.

In [4]:
# Let's get started with the first part stated above

In [5]:
# We need the predictions first for the train data - let's get those and later think about what we need next

In [6]:
# We need to arrange our train and validation data for this exercise too

In [7]:
# specify the directory where preprocessed data is stored
data_dir = 'exp_result'

In [8]:
cancer_gen_expr_model = tf.keras.models.load_model(os.path.join(data_dir,"cancer_gen_expr_model"))
cancer_gen_mut_model = tf.keras.models.load_model(os.path.join(data_dir, "cancer_gen_mut_model"))
cancer_dna_methy_model = tf.keras.models.load_model(os.path.join(data_dir, "cancer_dna_methy_model"))

2025-04-22 11:18:00.190880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-22 11:18:02.076893: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30960 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


WARNING tensorflow 2025-04-22 11:18:02,957:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


WARNING tensorflow 2025-04-22 11:18:03,205:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


WARNING tensorflow 2025-04-22 11:18:03,415:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


In [9]:
cancer_gen_expr_model.trainable = False
cancer_gen_mut_model.trainable = False
cancer_dna_methy_model.trainable = False

In [10]:
with open(os.path.join(data_dir, "drug_features.pickle"),"rb") as f:
        dict_features = pickle.load(f)

In [11]:
with open(os.path.join(data_dir, "norm_adj_mat.pickle"),"rb") as f:
        dict_adj_mat = pickle.load(f)

In [12]:
train_keep = pd.read_csv(os.path.join(data_dir, "train_y_data.csv"))
valid_keep = pd.read_csv(os.path.join(data_dir, "val_y_data.csv"))

In [13]:
train_keep.shape, valid_keep.shape

((7616, 3), (952, 3))

In [14]:
train_keep.columns = ["Cell_Line", "Drug_ID", "AUC"]
valid_keep.columns = ["Cell_Line", "Drug_ID", "AUC"]

In [15]:
samp_drug = valid_keep["Drug_ID"].unique()[-1]
samp_ach = np.array(valid_keep["Cell_Line"].unique()[-1])

In [16]:
print(samp_drug)
print(samp_ach)

Drug_1326
ACH-000828


In [17]:
train_gcn_feats = []
train_adj_list = []
for drug_id in train_keep["Drug_ID"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_adj_mat[drug_id])

In [18]:
len(train_gcn_feats), len(train_adj_list)

(7616, 7616)

In [19]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in valid_keep["Drug_ID"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_adj_mat[drug_id])

In [20]:
len(valid_gcn_feats), len(valid_adj_list)

(952, 952)

In [21]:
%%time
# reduce the values to float16
train_gcn_feats = np.array(train_gcn_feats).astype("float32")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float32")

train_adj_list = np.array(train_adj_list).astype("float32")
valid_adj_list = np.array(valid_adj_list).astype("float32")

CPU times: user 524 ms, sys: 789 ms, total: 1.31 s
Wall time: 1.32 s


In [22]:
# Define the data generators for both train and validation datasets. Let's use the train one now, and later think how we can use the validation data generator

In [23]:
train_data_gen = DataGenerator(train_gcn_feats, train_adj_list, train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["AUC"].values.reshape(-1,1), batch_size=32,  shuffle = False)

In [24]:
val_data_gen = DataGenerator(valid_gcn_feats, valid_adj_list, valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["AUC"].values.reshape(-1,1), batch_size=32,  shuffle = False)

In [25]:
# Okay, I think now once the model is loaded, we can just get the predictions

In [26]:
%%time
# I don't think we need a function for this, we should be able to just get it done in a for loop - we will also capture the predeictions for the validation data.

# location of the models
folder_path = 'bootstrap_results_all'

# name of the trained model
model_nm = 'DeepCDR_model'

all_train_predictions = []
all_valid_predictions = []
train_true = []
valid_true = []
# number of boostraps
B = 10

# start the for loop
for i in range(1, B + 1):
    # create the folder
    folder_nm = 'bootstrap_' + str(i)
    # joined path
    folder_loc = os.path.join(folder_path, folder_nm, model_nm)
    # load the model?
    model = tf.keras.models.load_model(folder_loc)
    # get the predictions on the train data
    y_train_preds, y_train_true = batch_predict(model, train_data_gen)
    y_val_preds, y_val_true = batch_predict(model, val_data_gen)
    all_train_predictions.append(y_train_preds)
    all_valid_predictions.append(y_val_preds)
    train_true.append(y_train_true)
    valid_true.append(y_val_true)

# Notice that we can add some lists to capture the predictions on the validation data as well, eventhough we have these stored, as this will save time - and I think we also need these on the test data inorder the compute the evaluation metrics, like the coverages and the widths.

2025-04-22 11:18:10.224025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
Predictions: 7616
True: 7616
Predictions: 952
True: 952
CPU times: user 2min 59s, sys: 13 s, total: 3min 12s
Wall time: 2min 53s


In [27]:
all_train_preds_array = np.array(all_train_predictions)

In [28]:
all_train_preds_array.shape

(10, 7616)

In [29]:
all_train_trues = np.array(train_true)

In [30]:
all_train_trues.shape

(10, 7616)

In [31]:
all_train_true_mean = np.mean(all_train_trues, axis = 0)

In [32]:
all_train_true_mean.shape

(7616,)

In [33]:
all_train_true_mean

array([0.7153, 0.9579, 0.413 , ..., 0.522 , 0.9436, 0.9835])

In [34]:
np.squeeze(train_keep["AUC"].values.reshape(-1,1))

array([0.7153, 0.9579, 0.413 , ..., 0.522 , 0.9436, 0.9835])

In [35]:
np.squeeze(train_keep["AUC"].values.reshape(-1,1)).shape

(7616,)

In [36]:
# these do look the same, should we do an np.mean?
np.mean(np.round(all_train_true_mean, 8) == np.round(np.squeeze(train_keep["AUC"].values.reshape(-1,1)), 8))

1.0

In [37]:
# Indeed the y true values match - sanity check complete

In [38]:
# Cool, so what next?

# I think computing the bootstrap quantities, the means and the variance of the predictions. Then we can work on training the NNe

In [39]:
# let's first compute the bootstrap means

train_bts_mean = np.mean(all_train_preds_array, axis = 0)

In [40]:
train_bts_mean.shape

(7616,)

In [41]:
# Get the same measure above for the validation data, as they are required for the validation data generator

In [42]:
all_valid_preds_array = np.array(all_valid_predictions)

In [43]:
all_valid_preds_array.shape

(10, 952)

In [44]:
all_valid_trues = np.array(valid_true)

In [45]:
all_valid_trues.shape

(10, 952)

In [46]:
all_valid_true_mean = np.mean(all_valid_trues, axis = 0)

In [47]:
all_valid_true_mean.shape

(952,)

In [48]:
all_valid_true_mean

array([0.5071, 0.6525, 0.8944, 0.8691, 0.8527, 0.7833, 0.977 , 0.3598,
       0.8045, 0.6042, 0.4953, 0.7821, 0.7977, 0.7289, 0.6565, 0.9151,
       0.8609, 0.8073, 0.4476, 0.8136, 0.8707, 0.9311, 0.8245, 0.8403,
       0.913 , 0.6812, 0.869 , 0.6503, 0.6457, 0.5061, 0.4772, 0.5123,
       0.913 , 0.8945, 0.8858, 0.8849, 0.8343, 0.8553, 0.8076, 0.8347,
       0.5389, 0.6081, 0.9519, 0.467 , 0.7841, 0.8478, 0.8789, 0.4217,
       0.8834, 0.9832, 0.7933, 0.3334, 0.6786, 0.8028, 0.8078, 0.6597,
       0.8299, 0.7734, 0.9431, 0.815 , 0.9919, 0.7359, 0.8435, 0.8259,
       0.9377, 0.8228, 0.8317, 0.789 , 0.4609, 0.8475, 0.8412, 0.8198,
       0.9699, 1.    , 0.6977, 0.7845, 0.8114, 0.3618, 0.7679, 0.9478,
       0.8955, 0.7926, 0.8577, 0.7597, 0.6948, 0.7047, 0.7249, 0.3023,
       0.8096, 0.5337, 0.558 , 0.9474, 0.7133, 0.9376, 0.9154, 0.9816,
       0.931 , 0.6853, 0.639 , 0.8035, 0.9536, 0.9689, 0.7831, 0.4605,
       0.7896, 0.836 , 0.9121, 0.8031, 0.8777, 0.8264, 0.8804, 0.8472,
      

In [49]:
np.squeeze(valid_keep["AUC"].values.reshape(-1,1))

array([0.5071, 0.6525, 0.8944, 0.8691, 0.8527, 0.7833, 0.977 , 0.3598,
       0.8045, 0.6042, 0.4953, 0.7821, 0.7977, 0.7289, 0.6565, 0.9151,
       0.8609, 0.8073, 0.4476, 0.8136, 0.8707, 0.9311, 0.8245, 0.8403,
       0.913 , 0.6812, 0.869 , 0.6503, 0.6457, 0.5061, 0.4772, 0.5123,
       0.913 , 0.8945, 0.8858, 0.8849, 0.8343, 0.8553, 0.8076, 0.8347,
       0.5389, 0.6081, 0.9519, 0.467 , 0.7841, 0.8478, 0.8789, 0.4217,
       0.8834, 0.9832, 0.7933, 0.3334, 0.6786, 0.8028, 0.8078, 0.6597,
       0.8299, 0.7734, 0.9431, 0.815 , 0.9919, 0.7359, 0.8435, 0.8259,
       0.9377, 0.8228, 0.8317, 0.789 , 0.4609, 0.8475, 0.8412, 0.8198,
       0.9699, 1.    , 0.6977, 0.7845, 0.8114, 0.3618, 0.7679, 0.9478,
       0.8955, 0.7926, 0.8577, 0.7597, 0.6948, 0.7047, 0.7249, 0.3023,
       0.8096, 0.5337, 0.558 , 0.9474, 0.7133, 0.9376, 0.9154, 0.9816,
       0.931 , 0.6853, 0.639 , 0.8035, 0.9536, 0.9689, 0.7831, 0.4605,
       0.7896, 0.836 , 0.9121, 0.8031, 0.8777, 0.8264, 0.8804, 0.8472,
      

In [50]:
np.squeeze(valid_keep["AUC"].values.reshape(-1,1)).shape

(952,)

In [51]:
# these do look the same, should we do an np.mean? - Do a sanity check
np.mean(np.round(all_valid_true_mean, 8) == np.round(np.squeeze(valid_keep["AUC"].values.reshape(-1,1)), 8))

1.0

In [52]:
valid_bts_mean = np.mean(all_valid_preds_array, axis = 0)

In [53]:
valid_bts_mean.shape

(952,)

In [54]:
# we also need the bootstrap variance

# let's use the same function as earlier - we cannot use this as is, as what we have now is a 2D array, and not a 3D one
def equation_6_model_variance(all_preds):
    all_vars = []
    for i in range(all_preds.shape[1]):
        var = (1/(all_preds.shape[0]  - 1))*np.sum(np.square(all_preds[:,i] - np.mean(all_preds[:,i])))
        all_vars.append(var)

    return np.array(all_vars, dtype= np.float32)

In [55]:
train_bts_variance = equation_6_model_variance(all_train_preds_array)

In [56]:
train_bts_variance.shape

(7616,)

In [57]:
train_bts_variance

array([0.00183184, 0.00109681, 0.00047159, ..., 0.00504019, 0.00060388,
       0.0024684 ], dtype=float32)

In [58]:
# how to alternatively compute the variance in one line
alt_train_bts_variance = np.var(all_train_preds_array, axis = 0, ddof = 1)

In [59]:
alt_train_bts_variance.shape

(7616,)

In [60]:
alt_train_bts_variance

array([0.00183184, 0.00109681, 0.00047159, ..., 0.00504019, 0.00060388,
       0.0024684 ], dtype=float32)

In [61]:
np.mean(np.round(train_bts_variance, 6) == np.round(alt_train_bts_variance, 6))

1.0

In [62]:
# Get the variances on the validation data

In [63]:
valid_bts_variance = equation_6_model_variance(all_valid_preds_array)

In [64]:
valid_bts_variance.shape

(952,)

In [65]:
valid_bts_variance

array([0.00111437, 0.00215758, 0.0016149 , 0.00110854, 0.00177893,
       0.0010395 , 0.00045934, 0.00218281, 0.00072607, 0.00161189,
       0.00129221, 0.00064204, 0.00067966, 0.00121964, 0.00054192,
       0.00165475, 0.00982931, 0.00098922, 0.00127981, 0.00106769,
       0.00047407, 0.00046066, 0.0012382 , 0.00408663, 0.004013  ,
       0.00104843, 0.00157014, 0.00091257, 0.00140339, 0.00090362,
       0.00096479, 0.00230562, 0.00060487, 0.0004463 , 0.00104181,
       0.00054409, 0.00093233, 0.00048041, 0.0004866 , 0.00109376,
       0.00160849, 0.00097167, 0.00090513, 0.00222176, 0.00079927,
       0.00217688, 0.00070226, 0.0027635 , 0.00018478, 0.00072723,
       0.00147327, 0.00253314, 0.00259756, 0.00222062, 0.00460852,
       0.00138656, 0.00166871, 0.00168314, 0.00151581, 0.00082051,
       0.00081417, 0.00152035, 0.00177849, 0.00065383, 0.00053532,
       0.00061589, 0.00129038, 0.00253621, 0.0022012 , 0.00195072,
       0.00133277, 0.00134491, 0.00129875, 0.00174996, 0.00114

In [66]:
# how to alternatively compute the variance in one line
alt_valid_bts_variance = np.var(all_valid_preds_array, axis = 0, ddof = 1)

In [67]:
alt_valid_bts_variance.shape

(952,)

In [68]:
alt_valid_bts_variance

array([0.00111437, 0.00215758, 0.0016149 , 0.00110854, 0.00177893,
       0.0010395 , 0.00045934, 0.00218281, 0.00072607, 0.00161189,
       0.00129221, 0.00064204, 0.00067966, 0.00121964, 0.00054192,
       0.00165475, 0.00982931, 0.00098922, 0.00127981, 0.00106769,
       0.00047407, 0.00046066, 0.0012382 , 0.00408663, 0.004013  ,
       0.00104843, 0.00157014, 0.00091257, 0.00140339, 0.00090362,
       0.00096479, 0.00230562, 0.00060487, 0.0004463 , 0.00104181,
       0.00054409, 0.00093233, 0.00048041, 0.0004866 , 0.00109376,
       0.00160849, 0.00097167, 0.00090513, 0.00222176, 0.00079927,
       0.00217688, 0.00070226, 0.0027635 , 0.00018478, 0.00072723,
       0.00147327, 0.00253314, 0.00259756, 0.00222062, 0.00460852,
       0.00138656, 0.00166871, 0.00168314, 0.00151581, 0.00082051,
       0.00081417, 0.00152035, 0.00177849, 0.00065383, 0.00053532,
       0.00061589, 0.00129038, 0.00253621, 0.0022012 , 0.00195072,
       0.00133277, 0.00134491, 0.00129875, 0.00174996, 0.00114

In [69]:
# sanity check ot see if the variances are inded correct
np.mean(np.round(valid_bts_variance, 6) == np.round(alt_valid_bts_variance, 6))

1.0

In [70]:
# sanity check for the means
catch_train_mean = []
for i in range(all_train_preds_array.shape[1]):
    computed_mean = np.mean(all_train_preds_array[:,i])
    catch_train_mean.append(computed_mean)

In [71]:
sanity_check_train_means = np.array(catch_train_mean)
np.mean(np.round(train_bts_mean,2) == np.round(sanity_check_train_means, 2))

1.0

In [72]:
# sanity check for the validation means
catch_valid_mean = []
for i in range(all_valid_preds_array.shape[1]):
    computed_mean = np.mean(all_valid_preds_array[:,i])
    catch_valid_mean.append(computed_mean)

In [73]:
sanity_check_valid_means = np.array(catch_valid_mean)
np.mean(np.round(valid_bts_mean, 3) == np.round(sanity_check_valid_means, 3))

1.0

In [74]:
# Okay, we have the variances and the means, now what?

In [75]:
# We need to compute the rsquare value mentioned in equation 10 of the paper -  we will do this for both train and validation data

In [76]:
train_keep["AUC"].values.shape

(7616,)

In [77]:
valid_keep["AUC"].values.shape

(952,)

In [78]:
# Compute r^2(x_i) for each bootstrap model
def compute_r_squared(y_true, y_pred, model_variance):
    residuals = (y_true - y_pred) ** 2 - model_variance
    return np.maximum(residuals, 0)

In [79]:
r_2_true_train = compute_r_squared(train_keep["AUC"].values, train_bts_mean, train_bts_variance)

In [80]:
r_2_true_train.shape

(7616,)

In [81]:
r_2_true_train

array([0.00820237, 0.00362897, 0.00079942, ..., 0.07589278, 0.00014988,
       0.01846294])

In [82]:
# are there any zeros?
np.min(r_2_true_train)

0.0

In [83]:
# Count the number of zeros
num_zeros_train = np.count_nonzero(r_2_true_train == 0)

In [84]:
num_zeros_train

3852

In [85]:
# That is a lot of zeros, but let's role with this number?

In [86]:
# What about teh validation data?

In [87]:
r_2_true_valid = compute_r_squared(valid_keep["AUC"].values, valid_bts_mean, valid_bts_variance)

In [88]:
r_2_true_valid.shape

(952,)

In [89]:
# are there any zeros?
np.min(r_2_true_valid)

0.0

In [90]:
# Count the number of zeros
num_zeros_valid = np.count_nonzero(r_2_true_valid == 0)

In [91]:
num_zeros_valid

357

#### We will define the custom loss function in equation 12 and also define a very basic model for the NNe -  Okay, this isn't as straight forward as how we would have liked, so we need to revisit this, and decide what the model structure we need for the NNe. Continue the work on this tomorrow?

In [92]:
# I'm thinking, maybe for now, we should not worry too much about the custom loss function, and proceed with a regular loss function

In [93]:
# Define the custom loss function as described in equation 12
def correct_custom_loss(r_true, r_pred):
    # first term in equation 12
    term_1 = tf.math.log(r_pred + 1)
    # define the second term
    term_2 = r_true/r_pred
    # cost function
    cost = 0.5 * tf.reduce_mean(term_1 + term_2)

    return cost

In [94]:
# # Step 3: Define the noise variance estimation network (Phase II)
# def create_noise_variance_nn(input_shape):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.InputLayer(input_shape=input_shape),
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(1, activation=tf.keras.activations.exponential)  # Output layer for variance prediction
#     ])
#     return model

In [95]:
# We cannot use the model above as we should be able to match all our features comin from the different omic types. Therefore we will use the same model that we used for training for training the NNe as well. The only difference will be, now instead of the original response y (AUC) we use the computed r_square.

In [96]:
# So, let's define the model as follows - with just one adjustment, make sure the last dense layer has an exponential activation function.

In [101]:
training = False
dropout1 = 0.10
dropout2 = 0.20
## get the model architecture
def deepcdrgcn_NNe(dict_features, dict_adj_mat, samp_drug, samp_ach, cancer_dna_methy_model, cancer_gen_expr_model, cancer_gen_mut_model, training = training, dropout1 = dropout1, dropout2 = dropout2):
    
    input_gcn_features = tf.keras.layers.Input(shape = (dict_features[samp_drug].shape[0], 75))
    input_norm_adj_mat = tf.keras.layers.Input(shape = (dict_adj_mat[samp_drug].shape[0], dict_adj_mat[samp_drug].shape[0]))
    mult_1 = tf.keras.layers.Dot(1)([input_norm_adj_mat, input_gcn_features])
    dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
    dense_out = dense_layer_gcn(mult_1)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)
    mult_2 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
    dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
    dense_out = dense_layer_gcn(mult_2)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)

    dense_layer_gcn = tf.keras.layers.Dense(100, activation = "relu")
    mult_3 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
    dense_out = dense_layer_gcn(mult_3)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)

    dense_out = tf.keras.layers.GlobalAvgPool1D()(dense_out)
    # All above code is for GCN for drugs

    # methylation data
    input_gen_methy1 = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    input_gen_methy = cancer_dna_methy_model(input_gen_methy1)
    input_gen_methy.trainable = False
    gen_methy_layer = tf.keras.layers.Dense(256, activation = "tanh")
    
    gen_methy_emb = gen_methy_layer(input_gen_methy)
    gen_methy_emb = tf.keras.layers.BatchNormalization()(gen_methy_emb)
    gen_methy_emb = tf.keras.layers.Dropout(dropout1)(gen_methy_emb, training = training)
    gen_methy_layer = tf.keras.layers.Dense(100, activation = "relu")
    gen_methy_emb = gen_methy_layer(gen_methy_emb)

    # gene expression data
    input_gen_expr1 = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    input_gen_expr = cancer_gen_expr_model(input_gen_expr1)
    input_gen_expr.trainable = False
    gen_expr_layer = tf.keras.layers.Dense(256, activation = "tanh")
    
    gen_expr_emb = gen_expr_layer(input_gen_expr)
    gen_expr_emb = tf.keras.layers.BatchNormalization()(gen_expr_emb)
    gen_expr_emb = tf.keras.layers.Dropout(dropout1)(gen_expr_emb, training = training)
    gen_expr_layer = tf.keras.layers.Dense(100, activation = "relu")
    gen_expr_emb = gen_expr_layer(gen_expr_emb)
    
    
    input_gen_mut1 = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    input_gen_mut = cancer_gen_mut_model(input_gen_mut1)
    input_gen_mut.trainable = False
    
    reshape_gen_mut = tf.keras.layers.Reshape((1, cancer_gen_mut_model(samp_ach).numpy().shape[0], 1))
    reshape_gen_mut = reshape_gen_mut(input_gen_mut)
    gen_mut_layer = tf.keras.layers.Conv2D(50, (1, 700), strides=5, activation = "tanh")
    gen_mut_emb = gen_mut_layer(reshape_gen_mut)
    pool_layer = tf.keras.layers.MaxPooling2D((1,5))
    pool_out = pool_layer(gen_mut_emb)
    gen_mut_layer = tf.keras.layers.Conv2D(30, (1, 5), strides=2, activation = "relu")
    gen_mut_emb = gen_mut_layer(pool_out)
    pool_layer = tf.keras.layers.MaxPooling2D((1,10))
    pool_out = pool_layer(gen_mut_emb)
    flatten_layer = tf.keras.layers.Flatten()
    flatten_out = flatten_layer(pool_out)
    x_mut = tf.keras.layers.Dense(100,activation = 'relu')(flatten_out)
    x_mut = tf.keras.layers.Dropout(dropout1)(x_mut)
    
    all_omics = tf.keras.layers.Concatenate()([dense_out, gen_methy_emb, gen_expr_emb, x_mut])
    x = tf.keras.layers.Dense(300,activation = 'tanh')(all_omics)
    x = tf.keras.layers.Dropout(dropout1)(x, training = training)
    x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=-1))(x)
    x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=1))(x)
    x = tf.keras.layers.Conv2D(filters=30, kernel_size=(1,150),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,2))(x)
    x = tf.keras.layers.Conv2D(filters=10, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
    x = tf.keras.layers.Conv2D(filters=5, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
    x = tf.keras.layers.Dropout(dropout1)(x, training = training)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(dropout2)(x, training = training)
    final_out_layer = tf.keras.layers.Dense(1, activation = tf.keras.activations.exponential)
    final_out = final_out_layer(x)
    simplecdr = tf.keras.models.Model([input_gcn_features, input_norm_adj_mat, input_gen_expr1,
                                   input_gen_methy1, input_gen_mut1], final_out)
    
    return simplecdr

In [102]:
# # Train function
# def train_model_nne(X_train, y_train, bootstrap_predictions, model_variance, model):
#     # Define custom loss inside model.compile using lambda
    
#     model.compile(optimizer='adam', 
#                   loss=lambda y_true, y_pred: correct_custom_loss(
#                       y_true, y_pred))  # y_true corresponds to r_true, and y_pred corresponds to r_pred
    
#     bootstrap_mean_predictions = np.squeeze(np.mean(bootstrap_predictions, axis = 0))
#     # The true residuals (r_true) can be computed outside and passed during training
#     r_true = compute_r_squared(y_train, bootstrap_mean_predictions, model_variance)
    
#     # Fit the model with the true residuals
#     model.fit(X_train, r_true, epochs=50, batch_size=32)

#     return model

In [103]:
# So, let's try and train this model now

In [108]:
# Let's define the model below

In [106]:
training = False
dropout1 = 0.10
dropout2 = 0.20
NNE_model = deepcdrgcn_NNe(dict_features, dict_adj_mat, samp_drug, samp_ach, cancer_dna_methy_model, cancer_gen_expr_model, cancer_gen_mut_model,  training = training, dropout1 = dropout1, dropout2 = dropout2)

In [107]:
NNE_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 223, 223)]   0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 223, 75)]    0           []                               
                                                                                                  
 dot (Dot)                      (None, 223, 75)      0           ['input_2[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 223, 256)     19456       ['dot[0][0]']                

In [109]:
# let's define the train and validation data generators, now with our output for the NNe and not the AUC value we have had earleir

In [110]:
train_gen_NNe = DataGenerator(train_gcn_feats, train_adj_list, train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), r_2_true_train, batch_size=32)

In [111]:
val_gen_NNe = DataGenerator(valid_gcn_feats, valid_adj_list, valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), r_2_true_valid, batch_size=32,  shuffle = False)

In [112]:
# compile the model
lr = 0.001
NNE_model.compile(loss = tf.keras.losses.MeanSquaredError(), 
                      # optimizer = tf.keras.optimizers.Adam(lr=1e-3),
                    optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False), 
                    metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [114]:
# fit the model   
batch_size = 32
generator_batch_size = 32
epoch_num = 150
patience_val = 10

In [116]:
# NNE_model.fit(train_gen_NNe, validation_data = val_gen_NNe, epochs = epoch_num, 
#          callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = patience_val, restore_best_weights=True, 
#                                                      mode = "min") ,validation_batch_size = generator_batch_size)

In [117]:
# We might need to copy and paste all these in a python script as we run into a graph execution error - Also we are currently rolling with the MSE loss -  we will adjust the loss later.